## Notebook de Exploración en Espacios de Estados

Se implementará un notebook con las siguientes funcionalidades:

  * Una función expansión, que será utilizada por todos los algoritmos
  * Una función por cada algoritmo de búsqueda: profundidad, amplitud, escalada, primero el mejor, branch & bound y A*
  * Las funciones deberán recibir como argumento otras funciones que permitan su uso para cualquier problema. Estas funciones dependientes del problema son:
    * sucesores(estado)
    * heuristico(estado, objetivo)
    * coste(camino)

Utilizar las funciones desarrolladas para
  * Encontrar el trayecto más corto entre dos estaciones del metro de Madrid
  * El problema de las vasijas
  * El 8-puzzle (opcional)

Ampliaciones opcionales:
  * Desarrollar una clase busqueda que incluya todos los métodos
  * Implementar otros problemas.

El profesor suministrará un esqueleto de notebook con algunas funciones implementadas.

**Importante**: para la coevaluación, sólo un miembro del grupo sube la práctica, aunque en el nombre del notebook debe aparecer el nombre de los dos.



#### Funciones auxiliares

In [2]:
def trace (msg): # auxiliar para escribir trazas por pantalla
	print(msg, end='\r', flush=True)

### Algunas cosas de python que necesitamos recordar

In [3]:
# una buena manera de representar grafos son los diccionarios
grafo = {} # inicializa un diccionario vacío
grafo['a'] = ('b', 'c') # asocia la tupla ('b', 'c') a la clave 'a'
grafo['a'] # obtiene el valor asociado a la clave 'a'

('b', 'c')

In [4]:
# para saber si un elemento está  en una lista
l = [1, 2, 3]
if 1 in l:
    print('sí está')

if 4 in l:
    print('sí está')

sí está


In [5]:
# para acceder al último elemento de una lista hay un "truco"
l[-1]

3

In [6]:
# para añadir un elemento al final de una lista (ojo, es destructiva, es decir, modifica la lista original)
l.append(4)
l

[1, 2, 3, 4]

In [7]:
# para concatenar dos listas (no modifica la lista original)
l2 = [4, 5, 6] + l
print(l2)
print(l)

l2 = [4,5,6] + [7]
print(l2)

[4, 5, 6, 1, 2, 3, 4]
[1, 2, 3, 4]
[4, 5, 6, 7]


In [8]:
# concatenación destructiva (destruye la lista original, pero es más eficiente)
l2 = [4, 5, 6]
l2.extend(l)
l2

[4, 5, 6, 1, 2, 3, 4]

In [9]:
# slicing de listas
l = [1, 2, 3, 4]
l[1:] # lista sin el primer elemento


[2, 3, 4]

In [10]:
# equivalente destructivo: pop
l.pop(0) # suprime el primer elemento (modifica la lista)
l

[2, 3, 4]

In [11]:
# sorted

l = [1, 4, 3, 2, 0, 8, 7, 5, 6]

l_ordenada = sorted(l)

print(l_ordenada)

l = [('a', 3), ('b', 1), ('d', 0), ('c', 6)]

def s2 (a):
    return a[1]

print(sorted(l, key=s2)) # key indica cómo acceder al item por el que se quiere ordenar dentro de una estructura compleja

print(sorted(l, key=lambda x: x[1]))

[0, 1, 2, 3, 4, 5, 6, 7, 8]
[('d', 0), ('b', 1), ('a', 3), ('c', 6)]
[('d', 0), ('b', 1), ('a', 3), ('c', 6)]


In [12]:
# Paso de funciones como argumento (Higher Order Functions)
# Sirven para "inyectar dependencias", es decir, para dejar parte del código sin implementar 
# y permitir que la implementación se pase más adelante a nuestro código
# Ejemeplo: imaginemos que queremos elevar todos los elementos de una lista al cuadrado

def todos_al_cuadrado (l):
    res = []
    for e in l:
        res.append(e**2)
    return res

l = [1,2,3,4]
l2 = todos_al_cuadrado(l)
print(l2)


[1, 4, 9, 16]


In [13]:
# pero, ¿y si quiero elevarlos al cubo? ¿Me tengo que hacer otra función?
# ¿Por qué no abstraer la operación a aplcar?

def aplica_a_todos (operacion, l):
    res = []
    for e in l:
        res.append(operacion(e))
    return res

l3 = aplica_a_todos(lambda x: x**3, l)
print(l3)

l3 = aplica_a_todos(lambda x: x-5, l)
print(l3)

[1, 8, 27, 64]
[-4, -3, -2, -1]


In [14]:
# esta función existe ya en python y se llama map

l4 = list(map(lambda x: x**3, l))
print(l4)

[1, 8, 27, 64]


### Grafo de ejemplo de las diapositivas de clase

In [15]:
# Grafo ejemplo de las diapositivas

# usaremos diccionarios

grafo = {}

grafo['a'] = ('b', 'c')
grafo['b'] = ('a', 'h')
grafo['c'] = ('a', 'd', 'e')
grafo['d'] = ('c', 'e', 'g')
grafo['e'] = ('c', 'd', 'f')
grafo['f'] = ('e')
grafo['g'] = ('d')
grafo['h'] = ('b')

In [16]:
def sucesores_grafo(nodo):
    return grafo[nodo]

### Expansión de caminos

Dado un camino, obtiene los sucesores del último nodo y crea un camino nuevo por cada uno de ellos, siempre que el camino resultante no se cíclico.

In [17]:
def expandir(camino, sucesores):
    nuevo = []
    final = camino[-1]
    for nodo in sucesores(final):
        if nodo not in camino:    
            nuevo.append(camino + [nodo])    
    
    return nuevo
  
expandir(['a'], sucesores_grafo)
# debe dar [['a','b'], ['a','c']

expandir(['a','c'], sucesores_grafo)
# debe dar [['a','c','d'], ['a','c','e']]

[['a', 'c', 'd'], ['a', 'c', 'e']]

### Algoritmos no informados

In [49]:
def profundidad(n0, nf, sucesores):
    '''n0 -> estado inicial
    nf -> estado final
    sucesores -> función que permite obtener los sucesores de un nodo
    La función debe devolver el camino solución, si lo hay, y None en otro caso
    '''
    if n0 == nf:
        return [n0]
    
    pila = [[n0]]  
    visitados = set([n0])
    
    while pila:
        camino_actual = pila.pop()
        nodo_actual = camino_actual[-1]
        
        for sucesor in sucesores(nodo_actual):
            if sucesor == nf:
                return camino_actual + [sucesor]
            
            if sucesor not in visitados:
                visitados.add(sucesor)
                pila.append(camino_actual + [sucesor])
    
    return None    

In [ ]:
def amplitud(n0, nf, sucesores):
    '''n0 -> estado inicial
    nf -> estado final
    sucesores -> función que permite obtener los sucesores de un nodo
    La función debe devolver el camino solución, si lo hay, y None en otro caso
    '''
    if n0 == nf:
        return [n0]
    
    cola = [[n0]]
    
    visitados = {n0}
    
    while cola:
        camino_actual = cola.pop(0)
        nuevos_caminos = expandir(camino_actual, sucesores)
        
        for nuevo_camino in nuevos_caminos:
            nuevo_nodo = nuevo_camino[-1]
            
            if nuevo_nodo == nf:
                return nuevo_camino
            
            if nuevo_nodo not in visitados:
                visitados.add(nuevo_nodo)
                cola.append(nuevo_camino)
    
    
    return None

Para los algoritmos informados, necesitamos definir un heurístico, tal que, dados dos nodos, actual y objetivo, devuelva la heurística entre esos dos nodos.
  * Para ello utilizaremos la distancia euclídea.
  * Necesitamos ampliar nuestra representación para incluir las coordenadas de cada nodo.
  * Una posibilidad es usar otro diccionario para las coordenadas.

In [20]:
posicion = {}
posicion['a'] = (0.0,0.0)
posicion['b'] = (1.2, -0.5)
posicion['c'] = (1.0, 0.8)
posicion['d'] = (2.0, 0.9)
posicion['e'] = (1.4, 0.3)
posicion['f'] = (2.5, 0.3)
posicion['g'] = (2.8, 0.5)
posicion['h'] = (0.0, -1.0)

def distancia_grafo(n1, n2):
    x1, y1 = posicion[n1]
    x2, y2 = posicion[n2]

    distancia = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
    
    return distancia

def coste_grafo(camino):
    if len(camino) < 2:
        return 0.0
    
    return sum(distancia_grafo(actual, siguiente) 
               for actual, siguiente in zip(camino[:-1], camino[1:]))

### Algoritmos informados

In [ ]:
def escalada(n0, nf, sucesores, h):
    '''n0 -> estado inicial
    nf -> estado final
    sucesores -> función que permite obtener los sucesores de un nodo
    h -> función que se aplica al nodo actual y al nodo final y devuelve la heurística de el nodo actual respecto del nodo final
    La función debe devolver el camino solución, si lo hay, y None en otro caso
    '''
    if n0 == nf:
        return [n0]
    
    CP = [[n0]]
    
    while CP and CP[0][-1] != nf:
        camino_actual = CP.pop(0)
        E = expandir(camino_actual, sucesores)
        
        if not E:
            continue
            
        O = sorted(E, key=lambda camino: h(camino[-1], nf))
        
        CP = O + CP
    
    if not CP:
        return None
    
    return CP[0]

In [22]:
def primero_el_mejor (n0, nf, sucesores, h):
	'''n0 -> estado inicial
	nf -> estado final
	sucesores -> función que permite obtener los sucesores de un nodo
	h -> función que se aplica al nodo actual y al nodo final y devuelve la heurística de el nodo actual respecto del nodo final
	La función debe devolver el camino solución, si lo hay, y None en otro caso
	'''
    # TODO

In [23]:
def poda (caminos): # si dos caminos que llegan al mismo nodo, elimina el más costoso, suponemos que están ordenados de menor a mayor coste
	pass

def branch_and_bound (n0, nf, sucesores, g):
	'''n0 -> estado inicial
	nf -> estado final
	sucesores -> función que permite obtener los sucesores de un nodo
	g -> función que se aplica a un camino y devuelve el coste total de dicho camino
	La función debe devolver el camino solución, si lo hay, y None en otro caso
	'''
	pass
	

In [24]:
def a_estrella (n0, nf, sucesores, g, h):
	'''n0 -> estado inicial
	nf -> estado final
	sucesores -> función que permite obtener los sucesores de un nodo
	g -> función que se aplica a un camino y devuelve el coste total de dicho camino
	h -> función que se aplica al nodo actual y al nodo final y devuelve la heurística de el nodo actual respecto del nodo final
	La función debe devolver el camino solución, si lo hay, y None en otro caso
	'''
	# TODO

### Búsqueda en el metro de madrid

La celda siguiente construye la estructura de líneas y estaciones del metro de Madrid.

La función sucesores recibe el nombre de una estación como argumento y devuelve las estaciones a las que se puede llegar en un paso.

In [25]:
# líneas de metro a 2025

todas = [['Pinar de Chamartín',
  'Bambú',
  'Chamartín',
  'Plaza de Castilla',
  'Valdeacederas',
  'Tetuán',
  'Estrecho',
  'Alvarado',
  'Cuatro Caminos',
  'Ríos Rosas',
  'Iglesia',
  'Bilbao',
  'Tribunal',
  'Gran Vía',
  'Sol',
  'Tirso de Molina',
  'Antón Martín',
  'Estación del Arte',
  'Atocha',
  'Menéndez Pelayo',
  'Pacífico',
  'Puente de Vallecas',
  'Nueva Numancia',
  'Portazgo',
  'Buenos Aires',
  'Alto del Arenal',
  'Miguel Hernández',
  'Sierra de Guadalupe',
  'Villa de Vallecas',
  'Congosto',
  'La Gavia',
  'Las Suertes',
  'Valdecarros'],
 ['Las Rosas',
  'Avenida de Guadalajara',
  'Alsacia',
  'La Almudena',
  'La Elipa',
  'Ventas',
  'Manuel Becerra',
  'Goya',
  'Príncipe de Vergara',
  'Retiro',
  'Banco de España',
  'Sevilla',
  'Sol',
  'Ópera',
  'Santo Domingo',
  'Noviciado',
  'San Bernardo',
  'Quevedo',
  'Canal',
  'Cuatro Caminos'],
 ['El Casar',
  'Villaverde Alto',
  'San Cristóbal',
  'Villaverde Bajo-Cruce',
  'Ciudad de los Ángeles',
  'San Fermín-Orcasur',
  'Hospital 12 de Octubre',
  'Almendrales',
  'Legazpi',
  'Delicias',
  'Palos de la Frontera',
  'Embajadores',
  'Lavapiés',
  'Sol',
  'Callao',
  'Plaza de España',
  'Ventura Rodríguez',
  'Argüelles',
  'Moncloa'],
 ['Argüelles',
  'San Bernardo',
  'Bilbao',
  'Alonso Martínez',
  'Colón',
  'Serrano',
  'Velázquez',
  'Goya',
  'Lista',
  'Diego de León',
  'Avenida de América',
  'Prosperidad',
  'Alfonso XIII',
  'Avenida de La Paz',
  'Arturo Soria',
  'Esperanza',
  'Canillas',
  'Mar de Cristal',
  'San Lorenzo',
  'Parque de Santa María',
  'Hortaleza',
  'Manoteras',
  'Pinar de Chamartín'],
 ['Alameda de Osuna',
  'El Capricho',
  'Canillejas',
  'Torre Arias',
  'Suanzes',
  'Ciudad Lineal',
  'Pueblo Nuevo',
  'Quintana',
  'El Carmen',
  'Ventas',
  'Diego de León',
  'Núñez de Balboa',
  'Rubén Darío',
  'Alonso Martínez',
  'Chueca',
  'Gran Vía',
  'Callao',
  'Ópera',
  'La Latina',
  'Puerta de Toledo',
  'Acacias',
  'Pirámides',
  'Marqués de Vadillo',
  'Urgel',
  'Oporto',
  'Vista Alegre',
  'Carabanchel',
  'Eugenia de Montijo',
  'Aluche',
  'Empalme',
  'Campamento',
  'Casa de Campo'],
 ['Laguna',
  'Carpetana',
  'Oporto',
  'Opañel',
  'Plaza Elíptica',
  'Usera',
  'Legazpi',
  'Arganzuela-Planetario',
  'Méndez Álvaro',
  'Pacífico',
  'Conde de Casal',
  'Sainz de Baranda',
  'O´Donnell',
  'Manuel Becerra',
  'Diego de León',
  'Avenida de América',
  'República Argentina',
  'Nuevos Ministerios',
  'Cuatro Caminos',
  'Guzmán el Bueno',
  'Vicente Aleixandre',
  'Ciudad Universitaria',
  'Moncloa',
  'Argüelles',
  'Príncipe Pío',
  'Puerta del Ángel',
  'Alto de Extremadura',
  'Lucero'],
 ['Hospital del Henares',
  'Henares',
  'Jarama',
  'San Fernando',
  'La Rambla',
  'Coslada Central',
  'Barrio del Puerto',
  'Estadio Metropolitano',
  'Las Musas',
  'San Blas',
  'Simancas',
  'García Noblejas',
  'Ascao',
  'Pueblo Nuevo',
  'Barrio de la Concepción',
  'Parque de las Avenidas',
  'Cartagena',
  'Avenida de América',
  'Gregorio Marañón',
  'Alonso Cano',
  'Canal',
  'Islas Filipinas',
  'Guzmán el Bueno',
  'Francos Rodríguez',
  'Valdezarza',
  'Antonio Machado',
  'Peñagrande',
  'Avenida de la Ilustración',
  'Lacoma',
  'Arroyofresno',
  'Pitis'],
 ['Nuevos Ministerios',
  'Colombia',
  'Pinar del Rey',
  'Mar de Cristal',
  'Feria de Madrid',
  'Aeropuerto T1-T2-T3',
  'Barajas',
  'Aeropuerto T-4'],
 ['Paco de Lucía',
  'Mirasierra',
  'Herrera Oria',
  'Barrio del Pilar',
  'Ventilla',
  'Plaza de Castilla',
  'Duque de Pastrana',
  'Pío XII',
  'Colombia',
  'Concha Espina',
  'Cruz del Rayo',
  'Avenida de América',
  'Núñez de Balboa',
  'Príncipe de Vergara',
  'Ibiza',
  'Sainz de Baranda',
  'Estrella',
  'Vinateros',
  'Artilleros',
  'Pavones',
  'Valdebernardo',
  'Vicálvaro',
  'San Cipriano',
  'Puerta de Arganda',
  'Rivas Urbanizaciones',
  'Rivas Futura',
  'Rivas Vaciamadrid',
  'La Poveda',
  'Arganda del Rey'],
 ['Hospital Infanta Sofía',
  'Reyes Católicos',
  'Baunatal',
  'Manuel de Falla',
  'Marqués de la Valdavia',
  'La Moraleja',
  'La Granja',
  'Ronda de la Comunicación',
  'Las Tablas',
  'Montecarmelo',
  'Tres Olivos',
  'Fuencarral',
  'Begoña',
  'Chamartín',
  'Plaza de Castilla',
  'Cuzco',
  'Santiago Bernabéu',
  'Nuevos Ministerios',
  'Gregorio Marañón',
  'Alonso Martínez',
  'Tribunal',
  'Plaza de España',
  'Príncipe Pío',
  'Lago',
  'Batán',
  'Casa de Campo',
  'Colonia Jardín',
  'Aviación Española',
  'Cuatro Vientos',
  'Joaquín Vilumbrales',
  'Puerta del Sur'],
 ['Plaza Elíptica',
  'Abrantes',
  'Pan Bendito',
  'San Francisco',
  'Carabanchel Alto',
  'La Peseta',
  'La Fortuna'],
 ['Puerta del Sur',
  'Parque de Lisboa',
  'Alcorcón Central',
  'Parque Oeste',
  'Universidad Rey Juan Carlos',
  'Móstoles Central',
  'Pradillo',
  'Hospital de Móstoles',
  'Manuela Malasaña',
  'Loranca',
  'Hospital de Fuenlabrada',
  'Parque Europa',
  'Fuenlabrada Central',
  'Parque de los Estados',
  'Arroyo Culebro',
  'Conservatorio',
  'Alonso de Mendoza',
  'Getafe Central',
  'Juan de la Cierva',
  'El Casar',
  'Los Espartales',
  'El Bercial',
  'El Carrascal',
  'Julián Besteiro',
  'Casa del Reloj',
  'Hospital Severo Ochoa',
  'Leganés Central',
  'San Nicasio'],
 ['Ópera', 'Príncipe Pío'],
 ['Pinar de Chamartín',
  'Fuente de la Mora',
  'Virgen del Cortijo',
  'Antonio Saura',
  'Álvarez de Villaamil',
  'Blasco Ibáñez',
  'María Tudor',
  'Palas de Rey',
  'Las Tablas'],
 ['Colonia Jardín',
  'Prado de la Vega',
  'Colonia de los Ángeles',
  'Prado del Rey',
  'Somosaguas Sur',
  'Somosaguas Centro',
  'Pozuelo Oeste',
  'Bélgica',
  'Dos Castillas',
  'Campus de Somosaguas',
  'Avenida de Europa',
  'Berna',
  'Estación de Aravaca'],
 ['Colonia Jardín',
  'Ciudad de la Imagen',
  'José Isbert',
  'Ciudad del Cine',
  'Cocheras',
  'Retamares',
  'Montepríncipe',
  'Ventorro del Cano',
  'Prado del Espino',
  'Cantabria',
  'Ferial de Boadilla',
  'Boadilla Centro',
  'Nuevo Mundo',
  'Siglo XXI',
  'Infante Don Luís',
  'Puerta de Boadilla']]

In [26]:
# para quitar acentos
a,b = 'áéíóúü','aeiouu'
trans = str.maketrans(a,b)

def buildModel (): # construye el grafo a partir de las listas de líneas
	# para cada estación busca: la anterior, la siguiente, los transbordos
	lista = ['#']
	for linea in todas:
		lista += linea
		lista += '#' # separa líneas entre sí

	model = {} # modelo que almacenará todos los sucesores de cada estación
	for e in lista: # para cada estación
		if e == '#':
			continue
		sucesores = []

		for i in range(len(lista)): # para cada aparición de la estación
			if e == lista[i]:
				if lista[i-1] != '#':
					sucesores.append(lista[i-1])
				if lista[i+1] != '#':
					sucesores.append(lista[i+1])

		# elimina acentos y mayúsculas de e
		e = e.lower()
		e = e.translate(trans)

		model[e] = sucesores
	return model

model = buildModel()

def sucesores_metro (estado):
	# elimina acentos y mayúsculas de estado, introduce algo de cómputo en cada llamada a sucesores
	estado = estado.lower()
	estado = estado.translate(trans)
	suc = model[estado]
	return list(set(suc)) # borra duplicados (puede haberlos en casos raros, como por ejemplo en 'Avenida de América')

In [27]:
sucesores_metro('rios rosas')

['Iglesia', 'Cuatro Caminos']

In [28]:
def coste_metro (camino):
	# TODO

def heuristico_metro (nodo, objetivo): # como no tenemos coordenadas, un heurístico posible es sumar puntos si el nodo y el objetivo no están en la misma línea
	# TODO

def printres (camino):
	print('')
	if not camino:
		print('No hay solución')
		return
	for n in camino:
		print(n)
	print(coste(camino))

origen = 'Pirámides'
destino = 'Ríos Rosas'

sol = a_estrella(origen, destino, sucesores_metro, coste_metro, heuristico_metro)
printres(sol)

''' debe dar
Pirámides
Acacias
Puerta de Toledo
La Latina
Ópera
Callao
Gran Vía
Tribunal
Bilbao
Iglesia
Ríos Rosas
Coste total: 14
Transbordos: 1
'''

IndentationError: expected an indented block after function definition on line 1 (1622999458.py, line 4)

### Problema de las vasijas

 * estado: [v1, v2] contenido de las vasijas 0 y 1
 * operaciones validas
   * llenar vasija(i), vaciar vasija(i), volcar(i, j)

Calcular soluciones para el menor número de pasos y para el menor gasto de agua

In [59]:
capacidad = [2, 5] # capacidad de las vasijas 0 y 1

# estados: (2,0)


def sucesores_vasijas(estado):
    v1, v2 = estado
    c1, c2 = capacidad
    sucesores = set()

    # Llenar la vasija 1
    if v1 < c1:
        sucesores.add((c1, v2))

    # Llenar la vasija 2
    if v2 < c2:
        sucesores.add((v1, c2))

    # Vaciar la vasija 1
    if v1 > 0:
        sucesores.add((0, v2))

    # Vaciar la vasija 2
    if v2 > 0:
        sucesores.add((v1, 0))

    # Volcar de la vasija 1 a la 2
    if v1 > 0 and v2 < c2:
        cantidad = min(v1, c2 - v2)
        sucesores.add((v1 - cantidad, v2 + cantidad))

    # Volcar de la vasija 2 a la 1
    if v2 > 0 and v1 < c1:
        cantidad = min(v2, c1 - v1)
        sucesores.add((v1 + cantidad, v2 - cantidad))

    return list(sucesores)


def gastoAgua(c): # calcula el coste como el gasto de agua de un camino
    if not c or len(c) < 2:
        return 0

    coste_total = 0
    
    for i in range(len(c) - 1):
        v1_anterior, v2_anterior = c[i]
        v1_actual, v2_actual = c[i+1]

        # Llenado de la vasija 1
        if v1_actual > v1_anterior and v2_actual == v2_anterior:
            coste_total += (v1_actual - v1_anterior)
        
        # Llenado de la vasija 2
        elif v2_actual > v2_anterior and v1_actual == v1_anterior:
            coste_total += (v2_actual - v2_anterior)
            
    return coste_total
 
 
def heuristico_vasijas(c):
    v1_actual, v2_actual = c[0]
    v1_objetivo, v2_objetivo = c[1]
    return abs(v1_actual - v1_objetivo) + abs(v2_actual - v2_objetivo)


estado_inicial = (0,0)
estado_final = (2, 3)
camino_menos_pasos = amplitud(estado_inicial, estado_final, sucesores_vasijas)

print(
    f'Los menores pasos para pasar de {estado_inicial} a {estado_final} es de ' 
    f'{len(camino_menos_pasos) - 1} pasos, siguiendo este camino: {camino_menos_pasos}'
    )


# Faltaría únicamente implementar la función de a* para 
# calcular el camino de menor gasto del agua 

# camino_menos_agua = a_estrella(
#     estado_inicial, 
#     estado_final, 
#     sucesores_vasijas, 
#     gastoAgua, 
#     heuristico_vasijas
# )

# print(
#     f'El número de pasos para pasar de {estado_inicial} a {estado_final} con el menor gasto de ' 
#     f'agua es de {len(camino_menos_agua) - 1} pasos, siguiendo este camino: {camino_menos_agua}'
#     )

Los menores pasos para pasar de (0, 0) a (2, 3) es de 2 pasos, siguiendo este camino: [(0, 0), (0, 5), (2, 3)]


### 8-puzzle (opcional)

![SNOWFALL](https://www.researchgate.net/profile/Ruo-Ando/publication/347300656/figure/fig1/AS:969204928901121@1608087870493/Initial-state-and-goal-state-of-8-puzzle_W640.jpg) 

  * Los estados se representarán como listas de listas.
  * La función sucesores, dado un estado, deberá devolver los estados a los que se puede pasar moviendo fichas adyacentes a la casilla vacía.
  * Como coste usar el menor número de pasos.
  * Idea para heurístico: la distancia de la posición actual de cada pieza a la posición que debe ocupar en el objetivo.

In [ ]:
objetivo = [[1, 2, 3], 
            [4, 5, 6], 
            [7, 8, 0]] # el 0 representa la casilla vacía

inicio = [[1, 0, 3], 
          [4, 2, 5], 
          [7, 8, 6]] # 3 pasos
#inicio = [[4, 1, 3], [7, 2, 5], [0, 8, 6]] # 6 pasos
#inicio = [[3, 1, 6], [2, 5, 4], [7, 8, 0]] # 14 pasos
#inicio = [[0, 4, 1], [3, 7, 2], [5, 8, 6]] # 18 pasos
#inicio = [[0, 2, 5], [3, 8, 6], [4, 1, 7]] # 22 pasos
#inicio = [[0, 3, 7], [1, 8, 5], [6, 4, 2]] # 22 pasos

In [1]:
def sucesores_puzzle(estado):
    pass
                
def heuristico_puzzle (e0, ef):
    pass

### Actividad opcional

Desarrollar una clase "Busqueda" que unifique dentro todos los algoritmos que hemos implementado en funciones independientes.

Para ello se especificarán en el constructor (__init__) las funciones dependientes del problema, es decir, sucesores, heurístico y coste, y de este modo no habrá que pasarlos como argumento a los algoritmos, sino sólo inicio y objetivo, quedando el código mucho más limpio.

Por ejemplo, el uso de esta clase sería algo así:

```python
b = busqueda(sucesores_metro, heuristico_metro, coste_metro)
b.aestrella('Pirámides', 'Conde de Casal')

```